# **AUTO**


In [2]:
import pandas as pd

df_a = pd.read_csv('/content/Classe_A_per_principio_attivo_15-10-2024.csv', encoding='latin1', delimiter=';')
df_h = pd.read_csv('/content/Classe_H_per_principio_attivo_15-10-2024.csv', encoding='latin1', delimiter=';')

df_a['Class'] = 'A'
df_h['Class'] = 'H'

combined = pd.concat([df_a, df_h], ignore_index=True)
combined.to_csv('combined.csv', index=False)

In [3]:
df = pd.read_csv('/content/combined.csv', encoding='latin1')
# Remove empty rows (rows where all elements are NaN)
df_cleaned = df.dropna(how='all')

# Remove duplicate rows
df_cleaned = df_cleaned.drop_duplicates()

# Save the cleaned dataset to a new CSV file
df_cleaned.to_csv('/content/combined-csv_cleaned.csv', index=False)

print("Dataset cleaned and saved as 'cleaned_combined_csv.csv'")

Dataset cleaned and saved as 'cleaned_combined_csv.csv'


In [4]:
# After cleaning and removing duplicates
df_cleaned.to_csv('/content/combined-csv_cleaned.csv', index=False)

# Reload cleaned dataset (optional, if you want to simulate a fresh start)
df_cleaned = pd.read_csv('/content/combined-csv_cleaned.csv', encoding='latin1')

# Filter rows by index (from 6437 to 6767 inclusive)
filtered_df = df_cleaned.loc[6435:6767]

# Save the filtered dataset
filtered_df.to_csv('/content/filtered_combined.csv', index=False)

print("Filtered dataset saved as 'filtered_combined.csv'")


# List of keywords or partial column names to keep (case-insensitive)
columns_to_keep = [
    'principio attivo',
    'descrizione gruppo',
    'denominazione e confezione',
    'titolare aic',
    'aic',
    'codice gruppo equivalenza',
    'class'
]

# Normalize column names in df to lowercase for matching
columns_lower = [col.lower() for col in filtered_df.columns]

# Normalize target columns to lowercase
columns_to_keep_lower = [col.lower() for col in columns_to_keep]

# Select columns from filtered_df that match (case-insensitive)
selected_columns = [filtered_df.columns[i] for i, col in enumerate(columns_lower) if col in columns_to_keep_lower]

# Create new DataFrame with selected columns
final_df = filtered_df[selected_columns]

# Save to CSV
final_df.to_csv('/content/final_filtered_combined.csv', index=False)

print("Final filtered dataset saved as 'final_filtered_combined.csv'")

Filtered dataset saved as 'filtered_combined.csv'
Final filtered dataset saved as 'final_filtered_combined.csv'


In [10]:
import requests
import pandas as pd  # if you use pandas elsewhere


HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36",
    "Accept": "application/json"
}

def get_codice_sis(aic_code):
    try:
        aic_int = int(float(aic_code))          # Convert code to int to remove decimals
        aic_code_str = str(aic_int).zfill(9)   # Then zero-pad to 9 digits
    except:
        aic_code_str = str(aic_code)            # If conversion fails, keep original as string


    search_url = f"https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query={aic_code_str}&spellingCorrection=true&page=0"
    print(f"Trying URL: {search_url}")
    try:
        response = requests.get(search_url, headers=HEADERS, timeout=5)
        print(f"Status code: {response.status_code}")  # <-- print HTTP response status
        print(f"Content-Type: {response.headers.get('Content-Type')}")  # <-- print content type
        print(f"Response snippet: {response.text[:200]}")  # <-- print first 200 chars of response text

        if "application/json" not in response.headers.get("Content-Type", ""):
            print(f"⚠️ Response is not JSON for AIC {aic_code}")
            return None

        data = response.json()

        if not data.get("data") or not data["data"].get("content"):
            print(f"⚠️ No data found for AIC {aic_code}")
            return None

        medicinale = data["data"]["content"][0].get("medicinale")
        if not medicinale:
            print(f"⚠️ No medicinale info for AIC {aic_code}")
            return None

        codice_sis = medicinale.get("codiceSis")
        return codice_sis

    except Exception as e:
        print(f"❌ Error for AIC {aic_code}: {e}")
        return None

# Assuming final_df is the DataFrame from previous steps with selected columns
# Rename the column that contains the AIC code to a consistent name for this code:
# You mentioned 'AIC Code (AIC)' is the exact column name, so let's rename it to 'AIC' for easier use
final_df = final_df.rename(columns={'AIC Code (AIC)': 'AIC'})

# Make sure 'AIC' column is string and zero-padded to 9 digits
final_df['AIC'] = final_df['AIC'].astype(str).str.zfill(9)

# Apply the API function to get codiceSis
final_df['codiceSis'] = final_df['AIC'].apply(get_codice_sis)

# Log all AIC codes that returned no codiceSis (None/NaN)
missing_aics = final_df[final_df['codiceSis'].isna()]['AIC'].tolist()
print(f"AIC codes with no results: {missing_aics}")

# Drop rows without codiceSis (None or NaN)
df_clean = final_df.dropna(subset=['codiceSis']).copy()

# Trim AIC code to first 5 digits
df_clean['AIC_trimmed'] = df_clean['AIC'].str[:5]

# Clean codiceSis to string integer form (remove decimal if present)
# Clean codiceSis to string integer form and zero-pad to 6 digits
def clean_codiceSis(code):
    try:
        return str(int(float(code))).zfill(6)  # zero-pad to 6 digits
    except:
        return str(code)

df_clean['codiceSis_clean'] = df_clean['codiceSis'].apply(clean_codiceSis)

# Trim AIC to 6 digits (the first 6 digits, as required by API)
df_clean['AIC_trimmed'] = df_clean['AIC'].str[:6]  # or .str[-6:] if last 6 digits needed

# Now build the URL column using the cleaned and trimmed codes
df_clean['URL'] = (
    "https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/organizzazione/"
    + df_clean['codiceSis_clean']
    + "/farmaci/"
    + df_clean['AIC_trimmed']
    + "/stampati?ts=RCP"
)

# Drop intermediate columns before saving or further processing
df_clean = df_clean.drop(columns=['codiceSis_clean', 'AIC_trimmed'])


# Save the final dataset with URLs
df_clean.to_csv('/content/final_filtered_dataset_with_urls.csv', index=False)

print(f"Dropped {len(final_df) - len(df_clean)} rows without codiceSis.")
print(f"Saved dataset with URLs, total rows: {len(df_clean)}")




Trying URL: https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=036160188&spellingCorrection=true&page=0
Status code: 200
Content-Type: application/json
Response snippet: {"status":200,"data":{"content":[{"id":"0000036778","principiAttiviIt":["OCTOCOG ALFA"],"confezioni":[{"idPackage":"73351","denominazionePackage":"500 UI POLVERE E SOLVENTE PER SOLUZIONE INIETTABILE 1
Trying URL: https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=036160152&spellingCorrection=true&page=0
Status code: 200
Content-Type: application/json
Response snippet: {"status":200,"data":{"content":[{"id":"0000036781","principiAttiviIt":["OCTOCOG ALFA"],"confezioni":[{"idPackage":"73354","denominazionePackage":"\"2000 UI POLVERE E SOLV.PER SOLUZ.INETTABILE-USO END
Trying URL: https://api.aifa.gov.it/aifa-bdf-eif-be/1.0.0/formadosaggio/ricerca?query=036160164&spellingCorrection=true&page=0
Status code: 200
Content-Type: application/json
Response snippet: {"status":200,"data

In [9]:
import os
import requests
from tqdm import tqdm
import pandas as pd

pdf_folder = '/content/aifa_pdfs'
os.makedirs(pdf_folder, exist_ok=True)

df = pd.read_csv('/content/final_filtered_dataset_with_urls.csv')

for idx, row in tqdm(df.iterrows(), total=len(df)):
    url = row['URL']
    aic_code = str(row['AIC']).split('.')[0]  # remove trailing '.0' if any

    try:
        response = requests.get(url, timeout=15)
        if response.status_code == 200:
            if response.content[:4] == b'%PDF':
                pdf_path = os.path.join(pdf_folder, f"{aic_code}.pdf")
                with open(pdf_path, 'wb') as f:
                    f.write(response.content)
            else:
                print(f"Skipping AIC {aic_code}: Content does not look like PDF. Possibly HTML or error page.")
        else:
            print(f"Skipping AIC {aic_code}: HTTP status {response.status_code}")

    except Exception as e:
        print(f"Error downloading AIC {aic_code}: {e}")


# After download, check which PDFs are missing

pdf_files = set(f.replace('.pdf', '') for f in os.listdir(pdf_folder) if f.endswith('.pdf'))
dataset_aics = set(df['AIC'].astype(str).str.split('.').str[0])  # remove trailing '.0' if any

missing_aics = dataset_aics - pdf_files

print(f"Number of missing PDFs: {len(missing_aics)}")
print("Missing AIC codes:")
print(missing_aics)




'''!pip install pymupdf
import os
import re
import fitz  # PyMuPDF
import pandas as pd
from tqdm import tqdm

# --- CONFIGURATION ---
pdf_folder = '/content/extracted_aifae.xlsx'

# --- Sections to extract ---
sections_to_extract = [
    "4.1 Therapeutic indications",
    "4.2 Posology and method of administration",
    "4.3 Contraindications",
    "4.4 Special warnings and precautions for use",
    "4.5 Interactions with other medicinal products",
    "4.6 Fertility, pregnancy and lactation",
    "4.7 Effects on ability to drive and use machines",
    "4.8 Undesirable effects",
    "4.9 Overdose",
    "6.2 Incompatibilities"
]

# Precompile regex for section headings
section_regex = re.compile(r"(4\.1|4\.2|4\.3|4\.4|4\.5|4\.6|4\.7|4\.8|4\.9|6\.2)\s+([^\n]+)", re.IGNORECASE)

# --- Function to extract section text ---
def extract_sections_from_text(text):
    matches = list(section_regex.finditer(text))
    section_dict = {}

    for i, match in enumerate(matches):
        section_num = match.group(1)
        section_title = match.group(2).strip()
        section_key = f"{section_num} {section_title}"
        section_key_clean = next((s for s in sections_to_extract if s.lower().startswith(section_num)), None)
        if not section_key_clean:
            continue
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        section_text = text[start:end].strip()
        section_dict[section_key_clean] = section_text

    # Add missing sections as "Not found"
    for s in sections_to_extract:
        if s not in section_dict:
            section_dict[s] = "Not found"

    return section_dict

# --- Process all PDFs ---
data = []
pdf_files = sorted(f for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf"))

for filename in tqdm(pdf_files, desc="Processing PDFs"):
    aic = filename.replace(".pdf", "").lstrip('0')  # <-- remove leading zeros from filename
    pdf_path = os.path.join(pdf_folder, filename)
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        for page in doc:
            full_text += page.get_text()
        doc.close()

        extracted = extract_sections_from_text(full_text)
        extracted["AIC"] = aic  # <-- store AIC without leading zeros
        data.append(extracted)

    except Exception as e:
        print(f"Error processing {filename}: {e}")

# --- Save to Excel ---
df = pd.DataFrame(data)
df = df[["AIC"] + sections_to_extract]  # Column order
df.to_excel(output_file, index=False)

print(f"\n✅ Extraction complete. File saved to:\n{output_file}")
'''





 19%|█▉        | 61/325 [01:28<05:37,  1.28s/it]

Skipping AIC 42192017: HTTP status 404


 20%|█▉        | 64/325 [01:32<05:09,  1.19s/it]

Skipping AIC 43038013: HTTP status 404


 20%|██        | 65/325 [01:32<04:32,  1.05s/it]

...

 81%|████████  | 263/325 [06:17<01:20,  1.29s/it]

Skipping AIC 44588147: HTTP status 404


100%|██████████| 325/325 [07:59<00:00,  1.47s/it]

Number of missing PDFs: 32
Missing AIC codes:
{'44588061', '43038013', '47997034', '50058116', '47997073', '44423010', '50058027', '44590166', '49444021', '47997010', '45512100', '44234250', '44399184', '47997059', '44399044', '44590091', '45512403', '44588147', '44234146', '44588022', '45512252', '43932084', '43932021', '50058066', '43932058', '47997097', '44590026', '49444033', '42192017', '49444019', '44234033', '44399119'}


'!pip install pymupdf\nimport os\nimport re\nimport fitz  # PyMuPDF\nimport pandas as pd\nfrom tqdm import tqdm\n\n# --- CONFIGURATION ---\npdf_folder = \'/content/extracted_aifae.xlsx\'\n\n# --- Sections to extract ---\nsections_to_extract = [\n    "4.1 Therapeutic indications",\n    "4.2 Posology and method of administration",\n    "4.3 Contraindications",\n    "4.4 Special warnings and precautions for use",\n    "4.5 Interactions with other medicinal products",\n    "4.6 Fertility, pregnancy and lactation",\n    "4.7 Effects on ability to drive and use machines",\n    "4.8 Undesirable effects",\n    "4.9 Overdose",\n    "6.2 Incompatibilities"\n]\n\n# Precompile regex for section headings\nsection_regex = re.compile(r"(4\\.1|4\\.2|4\\.3|4\\.4|4\\.5|4\\.6|4\\.7|4\\.8|4\\.9|6\\.2)\\s+([^\n]+)", re.IGNORECASE)\n\n# --- Function to extract section text ---\ndef extract_sections_from_text(text):\n    matches = list(section_regex.finditer(text))\n    section_dict = {}\n\n    for i, match 

In [12]:
!pip install PyMuPDF
pdf_folder = '/content/aifa_pdfs'
output_file = '/content/extracted_aifa.xlsx'  # output Excel file

import os
import re
import fitz  # PyMuPDF
import pandas as pd
from tqdm import tqdm


pdf_folder = '/content/aifa_pdfs'
pdf_files = [f for f in os.listdir(pdf_folder) if f.lower().endswith('.pdf')]
print(f"Number of PDF files found in folder: {len(pdf_files)}")

# Sections to extract
sections_to_extract = [
    "4.1 Therapeutic indications",
    "4.2 Posology and method of administration",
    "4.3 Contraindications",
    "4.4 Special warnings and precautions for use",
    "4.5 Interactions with other medicinal products",
    "4.6 Fertility, pregnancy and lactation",
    "4.7 Effects on ability to drive and use machines",
    "4.8 Undesirable effects",
    "4.9 Overdose",
    "6.2 Incompatibilities"
]

# Precompile regex for section headings
section_regex = re.compile(r"(4\.1|4\.2|4\.3|4\.4|4\.5|4\.6|4\.7|4\.8|4\.9|6\.2)\s+([^\n]+)", re.IGNORECASE)

def extract_sections_from_text(text):
    matches = list(section_regex.finditer(text))
    section_dict = {}

    for i, match in enumerate(matches):
        section_num = match.group(1)
        section_title = match.group(2).strip()
        section_key = f"{section_num} {section_title}"
        section_key_clean = next((s for s in sections_to_extract if s.lower().startswith(section_num)), None)
        if not section_key_clean:
            continue
        start = match.end()
        end = matches[i + 1].start() if i + 1 < len(matches) else len(text)
        section_text = text[start:end].strip()
        section_dict[section_key_clean] = section_text

    # Fill missing sections with "Not available"
    for s in sections_to_extract:
        if s not in section_dict:
            section_dict[s] = "Not available"

    return section_dict

# Process PDFs and extract sections
data = []
pdf_files = sorted(f for f in os.listdir(pdf_folder) if f.lower().endswith(".pdf"))

for filename in tqdm(pdf_files, desc="Processing PDFs"):
    aic = filename.replace(".pdf", "").lstrip('0')
    pdf_path = os.path.join(pdf_folder, filename)
    try:
        doc = fitz.open(pdf_path)
        full_text = ""
        for page in doc:
            full_text += page.get_text()
        doc.close()

        extracted = extract_sections_from_text(full_text)
        extracted["AIC"] = aic
        data.append(extracted)

    except Exception as e:
        print(f"❌ Error processing {filename}: {e}")  # <-- Add this line for debugging

# Save extracted sections to Excel
df = pd.DataFrame(data)
df = df[["AIC"] + sections_to_extract]  # specify column order
df.to_excel(output_file, index=False)

print(f"\n✅ Extraction complete. Saved to:\n{output_file}")


pdf_folder = '/content/aifa_pdfs'
pdf_files = set(f.replace('.pdf', '').lstrip('0') for f in os.listdir(pdf_folder) if f.endswith('.pdf'))

# Load your CSV dataset with all AICs
df_clean = pd.read_csv('/content/final_filtered_dataset_with_urls.csv', dtype={'AIC': str})

# Normalize AICs by stripping leading zeros to match file naming
dataset_aics = set(df_clean['AIC'].astype(str).str.lstrip('0'))

missing_aics = dataset_aics - pdf_files

print(f"Number of missing PDFs: {len(missing_aics)}")
print("Missing AIC codes:", missing_aics)



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 48.3 MB/s eta 0:00:00
Number of PDF files found in folder: 293


Processing PDFs: 100%|██████████| 293/293 [01:10<00:00,  4.18it/s]



✅ Extraction complete. Saved to:
/content/extracted_aifa.xlsx
Number of missing PDFs: 325
Missing AIC codes: {'44319299.0', '40870053.0', '47486117.0', '40223190.0', '44390021.0', '36027112.0', '44319426.0', '41011077.0', '36027011.0', '44234146.0', '27616022.0', '48491017.0', '41156011.0', '45710011.0', '40771115.0', '48464010.0', '44319034.0', '40333229.0', '47997010.0', '47486030.0', '38341018.0', '48464046.0', '44399044.0', '47488147.0', '44174187.0', '42192017.0', '48492021.0', '38965024.0', '28245114.0', '38983247.0', '26804082.0', '40429146.0', '37109131.0', '36892053.0', '45934130.0', '43038013.0', '43490135.0', '40223012.0', '39987019.0', '39949021.0', '50058027.0', '42368023.0', '46184038.0', '33638127.0', '26803104.0', '45645013.0', '44319162.0', '44411496.0', '42825012.0', '40089132.0', '36160152.0', '37665039.0', '40223202.0', '37110362.0', '40223048.0', '47486016.0', '38947139.0', '50058116.0', '42368035.0', '39100019.0', '36026060.0', '48819015.0', '27617024.0', '391000